In [1]:
import os
import zipfile
import urllib.request

In [2]:
url = "https://github.com/Shenggan/BCCD_Dataset/archive/refs/heads/master.zip"
filename = "bccd.zip"
dir = "bccd_dataset"

In [3]:
urllib.request.urlretrieve(url, filename)

('bccd.zip', <http.client.HTTPMessage at 0x1dc8d7880e0>)

In [4]:
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall(dir)

In [5]:

os.listdir(dir)

['BCCD_Dataset-master']